In [69]:
import numpy as np
import pandas as pd
import math
from pandas.api.types import is_string_dtype
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
np.random.seed(9999)




## Preprocessing of TElco Dataset


In [70]:
df = pd.read_csv("WA_Fn-UseC_-Telco-Customer-Churn.csv",na_values=[' ','?'])
rowNum=df.shape[0]


df.drop('customerID',inplace= True, axis=1)
df['TotalCharges'].fillna(value=df['TotalCharges'].mean(),inplace = True)



##One hot encoding
df = pd.get_dummies(df,drop_first=True) 




#standardize
ss=StandardScaler()
df.iloc[:,:]=ss.fit_transform(df.iloc[:,:])
# df=(df-df.min())/(df.max()-df.min())

dummyCol=np.ones(rowNum)#add all 1 to first column
df.insert(loc=0, column='dummy', value=dummyCol)

dataset=df.copy()



train, test = train_test_split(df, test_size=0.2, random_state=99)
df=train

Y_out = df.iloc[:, -1]  #i final output
Y_out = np.sign(Y_out)
df.drop(df.columns[-1], axis=1,inplace= True)  #drop the last column

# Dummy column full of ones at beginning of a matrix
rowNum=df.shape[0]
colNum= df.shape[1]
print(rowNum, colNum)


5634 31


In [86]:
print(Y_out.to_numpy())

[-1. -1. -1. ...  1. -1. -1.]


In [ ]:
print(df.head())

df.iloc[0]['SeniorCitizen']

In [78]:

def logisticRegression(data,result):
    checker=100000000
    oneMatrix= np.ones(rowNum)
    weight=np.random.rand(colNum)
#     weight= np.zeros(colNum)
    counter=0
    while (checker >0.5  and counter<50000 ):
        counter= counter +1
        alpha = 0.01
        tempDf= data.to_numpy()
        Yp = np.tanh(tempDf.dot(weight))
    #         Y = (df.iloc[:,-1]).to_numpy()
        Y= result
        M1 =1 - Yp **2
#         print(M1)
        M2= np.multiply( Y- Yp,M1)
        M2 = alpha * M2
        
        transposedDf= tempDf.transpose()
        r=transposedDf.dot(M2)
#         weight= weight+ np.multiply(r,alpha)
        weight = weight +r
        checker = np.sum((Y-Yp)**2)/(2* rowNum)
    print(checker)
#     print(weight)

    predicted= np.sign(np.tanh(tempDf.dot(weight)))

    acc= accuracy_score(Y_out, predicted)
    print("accuracy value " ,acc)
#     print(weight)
    return weight


logisticRegression(df,Y_out)

    

0.5
accuracy value  0.6957756478523252


array([-26.68      ,   7.585967  , -17.45672846,   9.71694101,
        -9.8164052 ,  -0.46621448,  -7.9899716 ,  -8.00516181,
         1.00251816,  -1.00251816,   2.03424738,  15.3285849 ,
       -11.57659546, -11.57659546,  -8.58395802, -11.57659546,
        -3.63655681, -11.57659546,  -3.77908358, -11.57659546,
        -7.83718398, -11.57659546,   2.94905642, -11.57659546,
         2.74846131,  -8.99942697, -15.03922262,   9.99224411,
        -6.44928535,  15.30699415,  -4.64992708])

In [ ]:
df.head()


In [ ]:
print(np.sign(Y_out))


In [77]:
def  WeightedMajority(h, z,data):
    h= np.array(h)
    z= np.array(z)
    print('z is ', z)
    print('h shape is ', h.shape)
    data= data.transpose()
    print(' data shape is ', data.shape)
    M1= h@data
    print(' m1 shape is ', M1.shape)
    p=np.tanh(M1)
    print(' p  is ',p)
    print(' P shape is ', p.shape)
    print ( '  Z shape is ',z.shape)
    predict=z@p
    print(predict)

    print(' predict shape is ', predict.shape)
    predict = np.sign(predict)
    return predict

# def  WeightedMajority(h, z,data):
#     print('z is ',z)
#     predict = np.zeros(len(data))
#     for i in range(len(h)):
#         predict= predict + z[i] * np.tanh(data.dot(h[i]))
#     print('predict is', predict)
#     predict= np.sign(predict)
#     for i in predict: 
#         if ( i > 0):
#             print('ok')
#     print(predict)
    
def confusionFunctions (  yTrue ,yPredict):
    tn, fp, fn, tp = confusion_matrix(yTrue,yPredict).ravel()
    print(tn, " ", fp, " ",fn , " ",tp)
    accuracy = (tp+ tn)/(tp+fp+fn+tn)
    print('accuracy is ',accuracy)
    
    
    sensitivity = tp/(tp+fn)
    print('sensitivity is ',sensitivity)
    
    specificity = tn/(tn+fp)
    print('specificity is ',specificity)
    
    precision = tp/(tp+fp)
    print('precision ',precision)
    
    fdr = fp/(fp+tp)
    print('fdr  ',fdr)
    
    f1_score=  2*tp/( 2*tp +fp + fn)
    print('f1 score ',f1_score)

    

    
    
    
def adaboost(iteration):
# print(df.iloc[0])
    iteration=20
    w=np.empty(rowNum)
    w.fill(1/rowNum)
    h=[]
    z=[]
    for k in range(iteration):
        df['ans'] = Y_out
#         print('w is ',w)

        data =df.sample(n= len(df),replace= True, weights=w,random_state=97)
        result = data['ans'].to_numpy()
        
        df.drop('ans',inplace= True, axis=1)
        data.drop('ans',inplace= True, axis=1)
#         print(data.shape)
        wL=logisticRegression(data,result)
        predicted= np.sign(np.tanh((data.to_numpy()).dot(wL)))
        
        
        error =0 
        for j in range(len(data)):
            if ( predicted[j] != result[j]) :
                error = error + w[j]
        if (error> 0.5 ):
            continue 
        h.append(wL)

#         print('w is  ',w,'error is ',error)

        for j in range(len(data)):
            if( predicted[j] == result[j]):
                w[j]=w[j] *(error)/(1-error)
                
        # Normalize W 
        s=np.sum(w)
        w= w/s
        print('error is ',error)
        # Finding z
        q=math.log2((1-error)/error)
#         print('q is ',q )
        z.append(q)
    predict= WeightedMajority(h,z,df)
    confusionFunctions(Y_out,predict)

        
adaboost(1)


0.5
accuracy value  0.4850905218317359
[-25.82         7.85877686 -17.93108763  10.41271695  -9.5636603
   0.70565737  -8.00079424  -8.06551378   1.35174408  -1.35174408
   1.48189643  15.41311735 -11.85897986 -11.85897986  -7.8686499
 -11.85897986  -3.25071516 -11.85897986  -2.73855213 -11.85897986
  -6.31598949 -11.85897986   3.48243181 -11.85897986   3.66458885
  -8.48273726 -15.71052138   9.83085832  -7.94557019  16.00832732
  -4.64239712]
error is  0.31132410365636237
0.5
accuracy value  0.5055023074192403
[-26.72         8.8244475  -15.87739543   9.81586671  -9.02020989
  -2.42592264  -8.69173314  -8.63391214   0.95560614  -0.95560614
   2.04816548  14.84017329 -11.288552   -11.288552    -7.36445183
 -11.288552    -2.82912708 -11.288552    -3.30802343 -11.288552
  -7.34891126 -11.288552     3.45348269 -11.288552     3.47806601
  -9.96238948 -13.40261926  10.24393972  -6.88991459  15.1448652
  -5.98491697]
error is  0.31863222795613205
0.5
accuracy value  0.49609513667021654
[-26.